hist, new, total, new/hist で特徴量作成

In [1]:
import warnings
import time
import sys
import datetime
import pickle
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', 500)

%matplotlib inline

In [2]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
def preprocessing(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    for col in ['category_3']:
        df[col] = df[col].map({'A':1, 'B':2,'C':3})
    return df

def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [4]:
# データの読み込み
path_new = '../../../../data/raw/new_merchant_transactions.csv'
path_hist = '../../../../data/raw/historical_transactions.csv'
path_train = '../../../../data/raw/train.csv'
path_test = '../../../../data/raw/test.csv'

new = pd.read_csv(path_new, parse_dates=['purchase_date'])
hist = pd.read_csv(path_hist ,parse_dates=['purchase_date'])

new = preprocessing(new)
hist = preprocessing(hist)

new = reduce_mem_usage(new)
hist = reduce_mem_usage(hist)

train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

target = train['target']
#del train['target']

Mem. usage decreased to 76.76 Mb (63.4% reduction)
Mem. usage decreased to 1193.84 Mb (61.6% reduction)


In [14]:
train = pd.read_csv(path_train, parse_dates=['first_active_month'])
test = pd.read_csv(path_test, parse_dates=['first_active_month'])

## train, test 処理

In [5]:
test_null_id = test[test.first_active_month.isna()].card_id.iloc[-1]
null_month = hist[hist.card_id == test_null_id].purchase_date.dt.strftime("%Y-%m-01").min()
test.fillna({'first_active_month':null_month},inplace=True)

In [6]:
for df in [test, train]:
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days

In [7]:
for df in [test,train]:
    df['first_active_month']  = (df['first_active_month'] - df['first_active_month'].min()).dt.days

## 特徴量作成

In [8]:
for df in [hist, new]:
    df['category_2'].fillna(1,inplace=True) # 1が最頻値
    df['category_3'].fillna(1,inplace=True) # 1が最頻値
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True) # 最頻値

In [9]:
# 承認フラグ　Yes,Noの数と比
# histのみ（newはYesのみのため）
authorized_flag = hist[['card_id','authorized_flag']].groupby(['card_id','authorized_flag']).agg(count=('card_id','count')).reset_index()

# カードIDごとの合計を計算
total_counts = authorized_flag.groupby('card_id')['count'].sum().reset_index()
total_counts = total_counts.rename(columns={'count': 'total_count'})

# 元のデータと合計を結合
authorized_flag = authorized_flag.merge(total_counts, on='card_id')

# 割合を計算
authorized_flag['ratio'] = authorized_flag['count'] / authorized_flag['total_count']
#hist[['card_id','authorized_flag']].groupby(['card_id']).agg(count=('authorized_flag','mean')).reset_index()

# 結果を整形
authorized_ratio = authorized_flag.pivot(index='card_id', columns='authorized_flag', values=['ratio','count']).reset_index().rename(columns={0: 'No', 1: 'Yes'})
authorized_ratio.columns = ['_'.join(col).strip() for col in authorized_ratio.columns.values]
authorized_ratio.rename(columns={'card_id_': 'card_id'},inplace=True)

In [10]:
hist_id_u = hist.groupby('card_id',as_index=False)\
    .agg(
            frequency = ('card_id','count'),
            amount_total = ('purchase_amount','sum'),
            amount_mean = ('purchase_amount','mean'),
            amount_max = ('purchase_amount','max'),
            amount_min = ('purchase_amount','min'),
            amount_median = ('purchase_amount','median'),
            merchant_category_nu = ('merchant_category_id','nunique'),
            merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
            city_nu = ('city_id','nunique'),
            city_mode = ('city_id',lambda x: x.mode()[0]),
            state_nu = ('state_id','nunique'),
            state_mode = ('state_id',lambda x: x.mode()[0]),
            date_min  = ('purchase_date','min'),
            date_max  = ('purchase_date','max'),
            installments_mean = ('installments','mean'),
            installments_max = ('installments','max'),
            installments_min = ('installments','min'),
            installments_median = ('installments','median'),
            installments_sum = ('installments','sum'),
            month_lag_mean = ('month_lag','mean'),
            month_lag_max = ('month_lag','max'),
            month_lag_min = ('month_lag','min'),
            month_lag_median = ('month_lag','median'),
            category_1_mod = ('category_1',lambda x: x.mode()[0]),
            category_2_mod = ('category_2',lambda x: x.mode()[0]),
            category_3_mod = ('category_3',lambda x: x.mode()[0]),
        )

In [11]:
hist_id_u = hist_id_u.merge(authorized_ratio,on='card_id')

In [12]:
hist_id_u['duration'] = (hist_id_u['date_max'] - hist_id_u['date_min']).dt.days
hist_id_u.date_max = (hist_id_u.date_max - hist_id_u.date_min.min()).dt.days
hist_id_u.date_min = (hist_id_u.date_min - hist_id_u.date_min.min()).dt.days

hist_id_u.merchant_category_mode = pd.to_numeric(hist_id_u.merchant_category_mode, errors='coerce')

hist_id_u.columns = ['hist_' + col if col != 'card_id' else col for col in hist_id_u.columns]

In [13]:
new_id_u = new.groupby('card_id',as_index=False)\
    .agg(
            frequency = ('card_id','count'),
            amount_total = ('purchase_amount','sum'),
            amount_mean = ('purchase_amount','mean'),
            amount_max = ('purchase_amount','max'),
            amount_min = ('purchase_amount','min'),
            amount_median = ('purchase_amount','median'),
            merchant_category_nu = ('merchant_category_id','nunique'),
            merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
            city_nu = ('city_id','nunique'),
            city_mode = ('city_id',lambda x: x.mode()[0]),
            state_nu = ('state_id','nunique'),
            state_mode = ('state_id',lambda x: x.mode()[0]),
            date_min  = ('purchase_date','min'),
            date_max  = ('purchase_date','max'),
            installments_mean = ('installments','mean'),
            installments_max = ('installments','max'),
            installments_min = ('installments','min'),
            installments_median = ('installments','median'),
            installments_sum = ('installments','sum'),
            month_lag_mean = ('month_lag','mean'),
            month_lag_max = ('month_lag','max'),
            month_lag_min = ('month_lag','min'),
            month_lag_median = ('month_lag','median'),
            category_1_mod = ('category_1',lambda x: x.mode()[0]),
            category_2_mod = ('category_2',lambda x: x.mode()[0]),
            category_3_mod = ('category_3',lambda x: x.mode()[0]),
        )

In [14]:
new_id_u['duration'] = (new_id_u['date_max'] - new_id_u['date_min']).dt.days
new_id_u.date_max = (new_id_u.date_max - new_id_u.date_min.min()).dt.days
new_id_u.date_min = (new_id_u.date_min - new_id_u.date_min.min()).dt.days

new_id_u.merchant_category_mode = pd.to_numeric(new_id_u.merchant_category_mode, errors='coerce')

new_id_u.columns = ['new_' + col if col != 'card_id' else col for col in new_id_u.columns]

In [15]:
total = pd.concat([hist,new],axis=0).reset_index(drop=True)

In [16]:
total_id_u = total.groupby('card_id',as_index=False)\
    .agg(
            frequency = ('card_id','count'),
            amount_total = ('purchase_amount','sum'),
            amount_mean = ('purchase_amount','mean'),
            amount_max = ('purchase_amount','max'),
            amount_min = ('purchase_amount','min'),
            amount_median = ('purchase_amount','median'),
            merchant_category_nu = ('merchant_category_id','nunique'),
            merchant_category_mode = ('merchant_category_id',lambda x: x.mode()[0]),
            city_nu = ('city_id','nunique'),
            city_mode = ('city_id',lambda x: x.mode()[0]),
            state_nu = ('state_id','nunique'),
            state_mode = ('state_id',lambda x: x.mode()[0]),
            date_min  = ('purchase_date','min'),
            date_max  = ('purchase_date','max'),
            installments_mean = ('installments','mean'),
            installments_max = ('installments','max'),
            installments_min = ('installments','min'),
            installments_median = ('installments','median'),
            installments_sum = ('installments','sum'),
            month_lag_mean = ('month_lag','mean'),
            month_lag_max = ('month_lag','max'),
            month_lag_min = ('month_lag','min'),
            month_lag_median = ('month_lag','median'),
            category_1_mod = ('category_1',lambda x: x.mode()[0]),
            category_2_mod = ('category_2',lambda x: x.mode()[0]),
            category_3_mod = ('category_3',lambda x: x.mode()[0]),
        )

In [17]:
total_id_u['duration'] = (total_id_u['date_max'] - total_id_u['date_min']).dt.days
total_id_u.date_max = (total_id_u.date_max - total_id_u.date_min.min()).dt.days
total_id_u.date_min = (total_id_u.date_min - total_id_u.date_min.min()).dt.days

total_id_u.merchant_category_mode = pd.to_numeric(total_id_u.merchant_category_mode, errors='coerce')

total_id_u.columns = ['total_' + col if col != 'card_id' else col for col in total_id_u.columns]

In [18]:
train = train.merge(hist_id_u, on='card_id',how='left')
test = test.merge(hist_id_u, on='card_id',how='left')

train = train.merge(new_id_u, on='card_id',how='left')
test = test.merge(new_id_u, on='card_id',how='left')

train = train.merge(total_id_u, on='card_id',how='left')
test = test.merge(total_id_u, on='card_id',how='left')

In [19]:
for df in [train, test]:
    df['hist_count_No'].fillna(0,inplace=True)
    df['hist_ratio_No'].fillna(0,inplace=True)

In [24]:
for df in [train, test]:
    df['ratio_amount_total'] = df['new_amount_total'] / df['hist_amount_total']
    df['ratio_amount_mean'] = df['new_amount_mean'] / df['hist_amount_mean']
    df['ratio_merchant_category_nu'] = df['new_merchant_category_nu'] / df['hist_merchant_category_nu']
    #df['ratio_installments_mean'] = df['new_installments_mean'] / df['hist_installments_mean']
    #df['ratio_installments_sum'] = df['new_installments_sum'] / df['hist_installments_sum']

In [29]:
test.describe()

C:\Users\kakeru.oshikiri.bt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\core\nanops.py:1487: RuntimeWarning: overflow encountered in cast
  return dtype.type(n)
C:\Users\kakeru.oshikiri.bt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\numpy\core\_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
C:\Users\kakeru.oshikiri.bt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\core\nanops.py:731: RuntimeWarning: invalid value encountered in scalar divide
  the_mean = the_sum / count if count > 0 else np.nan
C:\Users\kakeru.oshikiri.bt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pandas\core\nanops

,first_active_month,feature_1,feature_2,feature_3,elapsed_time,hist_frequency,hist_amount_total,hist_amount_mean,hist_amount_max,hist_amount_min,hist_amount_median,hist_merchant_category_nu,hist_merchant_category_mode,hist_city_nu,hist_city_mode,hist_state_nu,hist_state_mode,hist_date_min,hist_date_max,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_median,hist_installments_sum,hist_month_lag_mean,hist_month_lag_max,hist_month_lag_min,hist_month_lag_median,hist_category_1_mod,hist_category_2_mod,hist_category_3_mod,hist_ratio_No,hist_ratio_Yes,hist_count_No,hist_count_Yes,hist_duration,new_frequency,new_amount_total,new_amount_mean,new_amount_max,new_amount_min,new_amount_median,new_merchant_category_nu,new_merchant_category_mode,new_city_nu,new_city_mode,new_state_nu,new_state_mode,new_date_min,new_date_max,new_installments_mean,new_installments_max,new_installments_min,new_installments_median,new_installments_sum,new_month_lag_mean,new_month_lag_max,new_month_lag_min,new_month_lag_median,new_category_1_mod,new_category_2_mod,new_category_3_mod,new_duration,total_frequency,total_amount_total,total_amount_mean,total_amount_max,total_amount_min,total_amount_median,total_merchant_category_nu,total_merchant_category_mode,total_city_nu,total_city_mode,total_state_nu,total_state_mode,total_date_min,total_date_max,total_installments_mean,total_installments_max,total_installments_min,total_installments_median,total_installments_sum,total_month_lag_mean,total_month_lag_max,total_month_lag_min,total_month_lag_median,total_category_1_mod,total_category_2_mod,total_category_3_mod,total_duration,ratio_amount_total,ratio_amount_mean,ratio_merchant_category_nu
count,123623.000000,123623.00000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.0,123623.0,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.000000,110015.0,110015.0,110015.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.000000,123623.0,123623.0,123623.000000,110015.000000,110015.000000,110015.000000
mean,1902.883201,3.10926,1.741796,0.564377,381.116799,89.646360,-14.989707,-0.072851,36.608585,-0.730221,-0.622655,18.327010,520.658559,4.911473,132.199914,2.741432,10.627990,132.846266,377.479644,0.870227,4.340665,0.053744,0.628176,58.221496,-3.845194,-0.161831,-7.887812,-3.778059,0.063071,NaN,NaN,0.106988,0.893012,7.725464,81.920897,244.094473,6.756770,-3.709343,-0.519549,-0.000000,NaN,-0.577452,5.333264,312.230887,2.308222,128.031050,1.583584,10.738545,345.880253,375.275481,0.749674,1.488842,0.330028,0.667977,4.617225,1.477492,1.815789,1.159842,1.466109,0.024197,NaN,NaN,28.991910,95.659368,-18.290741,-0.107769,36.714069,-0.732763,-0.628641,20.690333,517.447765,5.438211,133.158886,2.937188,10.712529,132.846266,427.320482,0.851175,4.495474,-0.039653,0.609551,62.330472,-3.418250,1.588119,-7.887812,-3.411505,0.057384,NaN,NaN,293.901224,0.170961,1.079422,0.346279
std,292.557734,1.18911,0.749195,0.495840,292.557734,106.370249,1374.615845,27.466101,1348.625854,0.057023,0.424576,11.510162,240.475556,3

In [32]:
path_train_prepro = '../../../../data/processed/train_processed_20.csv'
path_test_preprp = '../../../../data/processed/test_processed_20.csv'

train.to_csv(path_train_prepro,index=False)
test.to_csv(path_test_preprp,index=False)